This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'w-9Qqxd7i5T3Jvo3Z_PG'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [3]:
# First, import the relevant modules
import requests
import json 

In [6]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse into the JSON structure that will be returned
quandl_sample = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2014-12-01&end_date=2014-12-01&api_key=w-9Qqxd7i5T3Jvo3Z_PG')

In [7]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(quandl_sample.json())

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2014-12-01', 'end_date': '2014-12-01', 'frequency': 'daily', 'data': [['2014-12-01', 22.1, 22.14, 21.82, 21.91, None, 21706.0, 477060.0, None, None, None]], 'collapse': None, 'order': None}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [8]:
# 1 Retrieve AFX_X stock data from quandl API
afx_x_2017 = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key=w-9Qqxd7i5T3Jvo3Z_PG')

In [9]:
# 2 Store JSON as Python Dictionary
a = afx_x_2017.json()

In [10]:
# Create empty lists to store data
a_opens = []
a_high = []
a_low = []
a_close = []
a_change = [0]    # First entry is a rate of change; set to 0
a_change_day = []
a_volume = []

In [12]:
# afx_dict['dataset_data']['data'][i][i] is as far as the nesting is needed

# Extract and store daily values through the yearly data

for i in range(len(a['dataset_data']['data'])):
    
    # Select data from relevant outer indices
    
    opening = a['dataset_data']['data'][i][1] 
    close = a['dataset_data']['data'][i][4]
    high = a['dataset_data']['data'][i][2] 
    low = a['dataset_data']['data'][i][3]    
    t_vol = a['dataset_data']['data'][i][6]
    
    # Check if data entries for that day exists for each of the parameters
    
    if isinstance(opening,float):
        a_opens.append(opening)
    if isinstance(close,float):
        a_close.append(close)
    if isinstance(high,float) and isinstance(low,float):    
        change_day = high - low
        a_change_day.append(change_day)
        
    # Change cannot be calculated based on the closing price of the first day
    
    if i != 0 : 
        change = a_close[i] - a_close[i-1]
        a_change.append(change)
        
    # Store the traded volume for each day. 
    
    if isinstance(t_vol,float):
        a_volume.append(t_vol)

In [15]:
# 3 Calculate what the highest and lowest opening prices were for the stock in this period.

print('The highest opening price is $%4.2f'% max(a_opens))
print('The lowest opening price is $%4.2f'% min(a_opens))

The highest opening price is $53.11
The lowest opening price is $34.00


In [16]:
# 4 What was the largest change in any one day (based on High and Low price)?

print('The largest change in any one day is $%4.2f'% max(a_change_day))

The largest change in any one day is $2.81


In [17]:
# 5 What was the largest change between any two days (based on Closing Price)?

print('The largest change in between 2 days based on closing price is $%4.2f'% max(a_change))

The largest change in between 2 days based on closing price is $2.56


In [18]:
# 6 What was the average daily trading volume during this year?

average_vol = sum(a_volume)/len(a_volume)
print('The average traded volume is $%9.2f'% average_vol)

The average traded volume is $ 89124.34


In [19]:
# 7 What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

median = sorted(a_volume)[int(len(a_volume)/2)]
print('Median volume is $%9.2f'% median)

Median volume is $ 76286.00
